In [2]:
# for generating 5 PC (principal component) pattern and PCA score

In [ ]:
import mne
from matplotlib import pyplot as plt
plt.style.use(['default'])
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import pickle
import os
import time
import math
import random
import argparse
import copy
import scipy
import scipy.io as sio
from scipy import signal
import mat73
import sklearn
import numpy as np
# from pytopo import topoplotIndie

In [4]:
'load relative band power'

PD_bp_re = sio.loadmat('RBDRP/03_PSD_EC/PD_bandpower_re.mat')
PD_bp_re = PD_bp_re['PD_bandpower_re']

HC_bp_re = sio.loadmat('RBDRP/03_PSD_EC/HC_bandpower_re_PD_matched.mat')
HC_bp_re = HC_bp_re['HC_bandpower_re']

In [5]:
'subj * band * voxel'
np.shape(PD_bp_re), np.shape(HC_bp_re)

((12, 5, 15002), (12, 5, 15002))

In [6]:
Xtrain_delta = np.log10(np.vstack((PD_bp_re[:,0,:], HC_bp_re[:,0,:])))
Xtrain_theta = np.log10(np.vstack((PD_bp_re[:,1,:], HC_bp_re[:,1,:])))
Xtrain_alpha = np.log10(np.vstack((PD_bp_re[:,2,:], HC_bp_re[:,2,:])))
Xtrain_beta1 = np.log10(np.vstack((PD_bp_re[:,3,:], HC_bp_re[:,3,:])))
Xtrain_beta2 = np.log10(np.vstack((PD_bp_re[:,4,:], HC_bp_re[:,4,:])))

In [7]:
np.shape(Xtrain_delta), np.shape(Xtrain_theta) # subj * voxel

((24, 15002), (24, 15002))

In [8]:
# PD vs. HC'
idx =np.shape(PD_bp_re)[0]*[1] + np.shape(HC_bp_re)[0]*[0]

In [9]:
nsubj = np.shape(Xtrain_delta)[0]

In [10]:
nsubj

24

In [11]:
# subtract subject mean
Xtrain_delta_rc = Xtrain_delta-np.repeat(Xtrain_delta.mean(axis=1),15002).reshape(nsubj,15002) #subtract subj mean (each row)
Xtrain_theta_rc = Xtrain_theta-np.repeat(Xtrain_theta.mean(axis=1),15002).reshape(nsubj,15002) #subtract subj mean (each row)
Xtrain_alpha_rc = Xtrain_alpha-np.repeat(Xtrain_alpha.mean(axis=1),15002).reshape(nsubj,15002) #subtract subj mean (each row)
Xtrain_beta1_rc = Xtrain_beta1-np.repeat(Xtrain_beta1.mean(axis=1),15002).reshape(nsubj,15002) #subtract subj mean (each row)
Xtrain_beta2_rc = Xtrain_beta2-np.repeat(Xtrain_beta2.mean(axis=1),15002).reshape(nsubj,15002) #subtract subj mean (each row)

In [12]:
# subtract group mean (each column)
SRP_delta = Xtrain_delta_rc-Xtrain_delta_rc.mean(axis=0)
SRP_theta = Xtrain_theta_rc-Xtrain_theta_rc.mean(axis=0)
SRP_alpha = Xtrain_alpha_rc-Xtrain_alpha_rc.mean(axis=0)
SRP_beta1 = Xtrain_beta1_rc-Xtrain_beta1_rc.mean(axis=0)
SRP_beta2 = Xtrain_beta2_rc-Xtrain_beta2_rc.mean(axis=0)

In [13]:
SRP_delta.shape #subj * voxel

(24, 15002)

In [14]:
cov_d = SRP_delta@SRP_delta.T #12*12 subj by subj cov matrix
cov_t = SRP_theta@SRP_theta.T #12*12 subj by subj cov matrix
cov_a = SRP_alpha@SRP_alpha.T #12*12 subj by subj cov matrix
cov_b1 = SRP_beta1@SRP_beta1.T #12*12 subj by subj cov matrix
cov_b2 = SRP_beta2@SRP_beta2.T #12*12 subj by subj cov matrix

In [15]:
np.shape(cov_d)

(24, 24)

In [17]:
from sklearn.decomposition import PCA
# band_score = (subject * Principal components)

pca1 = PCA(n_components=nsubj)
delta_score = pca1.fit_transform(cov_d)

pca2 = PCA(n_components=nsubj)
theta_score = pca2.fit_transform(cov_t)

pca3 = PCA(n_components=nsubj)
alpha_score = pca3.fit_transform(cov_a)

pca4 = PCA(n_components=nsubj)
beta1_score = pca4.fit_transform(cov_b1)

pca5 = PCA(n_components=nsubj)
beta2_score = pca5.fit_transform(cov_b2)

In [18]:
np.shape(delta_score)

(24, 24)

In [21]:
pca_temp = PCA(n_components = 5)
temp_score = pca_temp.fit_transform(cov_d)

In [22]:
np.shape(temp_score)

(24, 5)

In [20]:
# PC pattern
P_d = SRP_delta.T @ delta_score # (15002, 24) @ (24, 24) for PD
P_t = SRP_theta.T @ theta_score
P_a = SRP_alpha.T @ alpha_score
P_b1 = SRP_beta1.T @ beta1_score
P_b2 = SRP_beta2.T @ beta2_score

In [21]:
PC_delta_pattern = {'PC1':np.zeros(15002), 'PC2':np.zeros(15002), 'PC3':np.zeros(15002), 'PC4':np.zeros(15002), 'PC5':np.zeros(15002)}
PC_theta_pattern = {'PC1':np.zeros(15002), 'PC2':np.zeros(15002), 'PC3':np.zeros(15002), 'PC4':np.zeros(15002), 'PC5':np.zeros(15002)}
PC_alpha_pattern = {'PC1':np.zeros(15002), 'PC2':np.zeros(15002), 'PC3':np.zeros(15002), 'PC4':np.zeros(15002), 'PC5':np.zeros(15002)}
PC_beta1_pattern = {'PC1':np.zeros(15002), 'PC2':np.zeros(15002), 'PC3':np.zeros(15002), 'PC4':np.zeros(15002), 'PC5':np.zeros(15002)}
PC_beta2_pattern = {'PC1':np.zeros(15002), 'PC2':np.zeros(15002), 'PC3':np.zeros(15002), 'PC4':np.zeros(15002), 'PC5':np.zeros(15002)}

In [22]:
np.shape(P_d)

(15002, 24)

In [23]:
np.shape(delta_score)

(24, 24)

In [24]:
PC = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5']

for idx, pc in enumerate(PC):
    PC_delta_pattern[pc] = P_d[:,idx]
    PC_theta_pattern[pc] = P_t[:,idx] 
    PC_alpha_pattern[pc] = P_a[:,idx]
    PC_beta1_pattern[pc] = P_b1[:,idx]
    PC_beta2_pattern[pc] = P_b2[:,idx]

In [26]:
sio.savemat('PD_'+'PC_delta_pattern.mat', PC_delta_pattern)
sio.savemat('PD_'+'PC_theta_pattern.mat', PC_theta_pattern)
sio.savemat('PD_'+'PC_alpha_pattern.mat', PC_alpha_pattern)
sio.savemat('PD_'+'PC_beta1_pattern.mat', PC_beta1_pattern)
sio.savemat('PD_'+'PC_beta2_pattern.mat', PC_beta2_pattern)

In [27]:
np.shape(SRP_delta)

(24, 15002)

In [28]:
score = {}

In [29]:
score['delta'] = delta_score
score['theta'] = theta_score
score['alpha'] = alpha_score
score['beta1'] = beta1_score
score['beta2'] = beta2_score

In [30]:
sio.savemat('PD_'+'score.mat', score)

In [31]:
SRP = {}

In [32]:
SRP['SRP_delta'] = SRP_delta
SRP['SRP_theta'] = SRP_theta
SRP['SRP_alpha'] = SRP_alpha
SRP['SRP_beta1'] = SRP_beta1
SRP['SRP_beta2'] = SRP_beta2

In [33]:
sio.savemat('PD_'+'SRP.mat', SRP)

In [34]:
GMP_ref = {}
# Xtrain_delta_rc.mean(axis=0).shape --> (15002,)

In [36]:
GMP_ref['GMP_ref_delta'] = Xtrain_delta_rc.mean(axis=0)
GMP_ref['GMP_ref_theta'] = Xtrain_theta_rc.mean(axis=0)
GMP_ref['GMP_ref_alpha'] = Xtrain_alpha_rc.mean(axis=0)
GMP_ref['GMP_ref_beta1'] = Xtrain_beta1_rc.mean(axis=0)
GMP_ref['GMP_ref_beta2'] = Xtrain_beta2_rc.mean(axis=0)

In [37]:
sio.savemat('PD_'+'GMP_ref.mat', GMP_ref)